In [ ]:
# ! pip install -r ../requirements-dev.txt -q

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path
import h3
from math import radians, cos, sin, asin, sqrt, degrees, atan2

## Собираем табличку с трейн данными

In [ ]:
data_dir = Path("..\data")

In [ ]:
transactions_df = pd.read_parquet(data_dir / "transactions.parquet").drop_duplicates().rename(columns={'h3_09': 'h3_09_trans'})
# Заполняем std = 0 где одна транзакция
transactions_df['std'].fillna(0, inplace=True)

In [ ]:
transactions_df.head()

In [ ]:
target_df = pd.read_parquet(data_dir / "target.parquet").drop_duplicates().rename(columns={'h3_09': 'h3_09_cash'})
target_df.head()

In [ ]:
# Определяем локации банкоматов и терминалов, мерджим банкоматы к транзакциям по клиенту
trans_cash_df = transactions_df.merge(target_df, on='customer_id', how='left')

used_locations_trans = trans_cash_df[["h3_09_trans"]].drop_duplicates()
used_locations_cash = trans_cash_df[["h3_09_cash"]].drop_duplicates()

used_locations_trans[['lat_trans', 'lon_trans']] = used_locations_trans['h3_09_trans'].apply(lambda x: pd.Series(h3.h3_to_geo(x)))
used_locations_cash[['lat_cash', 'lon_cash']] = used_locations_cash['h3_09_cash'].apply(lambda x: pd.Series(h3.h3_to_geo(x)))

trans_cash_df = trans_cash_df.merge(used_locations_trans, on=["h3_09_trans"], how='left')\
                             .merge(used_locations_cash, on=["h3_09_cash"], how='left')

In [ ]:
# original formula from  http://www.movable-type.co.uk/scripts/latlong.html
def distance_haversine(p1, p2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    Haversine
    formula: 
        a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
                        _   ____
        c = 2 ⋅ atan2( √a, √(1−a) )
        d = R ⋅ c

    where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
            note that angles need to be in radians to pass to trig functions!
    """
    def validate_point(p):
        lat, lon = p
        assert -90 <= lat <= 90, "bad latitude"
        assert -180 <= lon <= 180, "bad longitude"
    
    lat1, lon1 = p1
    lat2, lon2 = p2
    for p in [p1, p2]:
        validate_point(p)

    R = 6371 # km - earths's radius

    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # haversine formula 
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) # 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c
    return d

In [ ]:
# lat lon делаем одной колонкой для удобства
trans_cash_df['lat_lon_trans'] = trans_cash_df[['lat_trans','lon_trans']].values.tolist()
trans_cash_df['lat_lon_cash'] = trans_cash_df[['lat_cash','lon_cash']].values.tolist()

In [ ]:
trans_cash_df.head()

In [ ]:
# Считаем расстояние от бонкоматов где чел снимал деньги до его транзакций
# Может помочь для определения точек транзакций для которых нужна наличка
trans_cash_df['trans_cash_distance'] = trans_cash_df.apply(lambda x: distance_haversine(x['lat_lon_trans'], x['lat_lon_cash']), axis=1)

In [ ]:
# trans_cash_df.to_csv('transactions_cash.csv')

In [ ]:
trans_cash_df.head(10)

## Посмотрим на выбранного клиента

In [ ]:
customer_id = 14235
trans_cash_df[(trans_cash_df["customer_id"]==customer_id)]
# Чел в разные дни снимает в одном и том же банкомате (только однажды решил снять в 8911aa78c93ffff)

In [ ]:
(transactions_df.groupby(['customer_id'], as_index=False)['h3_09_trans'].count()
                                         .sort_values(by='h3_09_trans', ascending=False))

In [ ]:
def plot_trans_and_cash(df: pd.DataFrame, customer_id: int = None) -> None:
    """Plots map of transactions and cash withdraw of particular client or all clients"""
    plot_df = df.copy()

    if customer_id:
        plot_df = plot_df.query('customer_id == @customer_id')
    plot_cash_df = plot_df[['lat_cash', 'lon_cash', 'datetime_id']].drop_duplicates().rename(columns={'lat_cash': 'lat', 'lon_cash': 'lon'})
    plot_cash_df['target'] = 1
    plot_trans_df = plot_df[['lat_trans', 'lon_trans', 'datetime_id']].drop_duplicates().rename(columns={'lat_trans': 'lat', 'lon_trans': 'lon'})
    plot_trans_df['target'] = 0
    plot_df = pd.concat([plot_trans_df, plot_cash_df])

    # Добавим разброс, чтобы все было видно на карте
    # plot_df['lat'] += np.random.normal(0, 0.0005, len(plot_df))
    # plot_df['lon'] += np.random.normal(0, 0.0005, len(plot_df))
    color_mapping = {0: 'blue', 1: 'red'}
    color_sequence = plot_df['target'].apply(lambda x:color_mapping[x]).values
    fig = px.scatter_mapbox(plot_df,
                            lat='lat',
                            lon='lon',
                            zoom=10, 
                            hover_data=["datetime_id"],
                            color=plot_df['target'], color_discrete_sequence=color_sequence,
                           mapbox_style="open-street-map",
                           height=1000)
    # fig.update_layout(title='Plot of Points', hovermode='closest')
    fig.show()

In [ ]:
plot_trans_and_cash(trans_cash_df, 70785)

## Еще файлы

In [ ]:
# список всех 1658 геолокаций, где возможно снятие наличности, нужно разметить эти локации в решении
# тут 77 локаций, которых нет в transactions_df.query("target==1").h3_09.unique()
with open(data_dir / "hexses_target.lst", "r") as file:
    file_contents = file.read()
hexses_target = file_contents.split("\n")

In [ ]:
# бесполезный файл, список всех уникальных h3_09 из transactions_df (из всего 8154)
with open(data_dir / "hexses_data.lst", "r") as file:
    file_contents = file.read()
hexses_data = file_contents.split("\n")

In [ ]:
moscow_df = pd.read_parquet(data_dir / "moscow.parquet")

In [ ]:
moscow_df.head()

In [ ]:
for tag in moscow_df[~moscow_df.tags.isna()].tags.iloc[:20]:
    print(tag)

In [ ]:
fig = px.scatter_mapbox(moscow_df.iloc[:10000], lat='lat', lon='lon', hover_name='id', zoom=10, hover_data=["tags"])
fig.update_layout(mapbox_style="open-street-map", height=1000)
fig.update_layout(title='Plot of Points', hovermode='closest')
fig.show()